In [70]:
import json 
import pandas as pd 

with open('/Users/eiphyusinn/Desktop/Data_science_project/data/properties.json', 'r') as f:
    data = json.load(f)
    data_df = pd.DataFrame(data)
data_df.head(2)


,title,price,property_id,details
0,Spacious Penthouse in Mayangone,"$4,500 /Month",150515,"{'property_type': 'Penthouse', 'township': 'Ma..."
1,Modern 3 Bed Crystal Residence,"$2,300 /Month",150469,"{'property_type': 'Condo', 'township': 'Kamayu..."


In [71]:
data_df = pd.json_normalize(data) 
data_df.head(2)

,title,price,property_id,details.property_type,details.township,details.bedrooms,details.property_size
0,Spacious Penthouse in Mayangone,"$4,500 /Month",150515,Penthouse,Mayangone,3,6000 ft²
1,Modern 3 Bed Crystal Residence,"$2,300 /Month",150469,Condo,Kamayut,3,2000 ft²


In [72]:
df = data_df[['details.property_type', 'details.township', 'details.bedrooms', 'details.property_size','price']]
df.columns = ['property_type', 'township', 'bedrooms', 'property_size', 'price']

df.head(2)

,property_type,township,bedrooms,property_size,price
0,Penthouse,Mayangone,3,6000 ft²,"$4,500 /Month"
1,Condo,Kamayut,3,2000 ft²,"$2,300 /Month"


In [73]:
df.dtypes

property_type    object
township         object
bedrooms         object
property_size    object
price            object
dtype: object

In [74]:
df.isna().sum()

property_type     0
township          0
bedrooms         70
property_size     7
price             0
dtype: int64

In [75]:
df = df.copy() 
df.dropna(subset=['property_size'], inplace=True)
df['bedrooms'] = df['bedrooms'].fillna('0')
df.isna().sum()

property_type    0
township         0
bedrooms         0
property_size    0
price            0
dtype: int64

In [76]:
df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce').fillna(0).astype(int)

In [77]:
def parse_size(x):
    x = x.replace("ft²", "").strip()
    if "to" in x:
        nums = [int(n.strip()) for n in x.split("to")]
        return sum(nums) // len(nums)  
    try:
        return int(x)
    except:
        return 0 

df['property_size'] = df['property_size'].apply(parse_size)


In [83]:
# Remove rows where price is "Contact for price" or contains "for sale"
df = df[~df['price'].str.strip().str.lower().isin(['contact for price'])] 
df = df[~df['price'].str.strip().str.lower().str.contains('for sale')]  

def parse_price(x):
    x = x.replace("/Month", "").replace("$", "").replace(",", "").strip()
    if '-' in x:
        nums = [int(n.strip()) for n in x.split('-')]
        return sum(nums) // len(nums) 
    return int(x)

df['price'] = df['price'].apply(parse_price)


In [84]:
df.head(2)

,property_type,township,bedrooms,property_size,price
0,Penthouse,Mayangone,3,6000,4500
1,Condo,Kamayut,3,2000,2300
